1. Scala's real-world project repository data
With almost 30k commits and a history spanning over ten years, Scala is a mature programming language. It is a general-purpose programming language that has recently become another prominent language for data scientists.

Scala is also an open source project. Open source projects have the advantage that their entire development histories -- who made changes, what was changed, code reviews, etc. -- are publicly available.

We're going to read in, clean up, and visualize the real world project repository of Scala that spans data from a version control system (Git) as well as a project hosting site (GitHub). We will find out who has had the most influence on its development and who are the experts.

The dataset we will use, which has been previously mined and extracted from GitHub, is comprised of three files:

pulls_2011-2013.csv contains the basic information about the pull requests, and spans from the end of 2011 up to (but not including) 2014.
pulls_2014-2018.csv contains identical information, and spans from 2014 up to 2018.
pull_files.csv contains the files that were modified by each pull request.

In [12]:
# Importing pandas
import pandas as pd
# ... YOUR CODE FOR TASK 1 ...

# Loading in the data
pulls_one = pd.read_csv('pulls_2011-2013.csv')
pulls_two = pd.read_csv('pulls_2014-2018.csv')
pulls = pd.read_csv('pulls.csv')
pull_files = pd.read_csv('pull_files.csv')

In [13]:
pull_files.head()

,pid,file
0,163314316,test/files/pos/t5638/Among.java
1,163314316,test/files/pos/t5638/Usage.scala
2,163314316,test/files/pos/t9291.scala
3,163314316,test/files/run/t8348.check
4,163314316,test/files/run/t8348/TableColumn.java


In [14]:
pulls['date'] = pd.to_datetime(pulls['date'],utc=True)

In [15]:
pulls.head()

,pid,user,date
0,163314316,hrhino,2018-01-16 23:29:16+00:00
1,163061502,joroKr21,2018-01-15 23:44:52+00:00
2,163057333,mkeskells,2018-01-15 23:05:06+00:00
3,162985594,lrytz,2018-01-15 15:52:39+00:00
4,162838837,zuvizudar,2018-01-14 19:16:16+00:00


In [16]:
# Merge the two DataFrames
data = pulls.merge(pull_files,on='pid')

In [17]:
joined_pr = pull_files.merge(pull_files, on='pid')

In [19]:
joined_pr.head()

,pid,file_x,file_y
0,163314316,test/files/pos/t5638/Among.java,test/files/pos/t5638/Among.java
1,163314316,test/files/pos/t5638/Among.java,test/files/pos/t5638/Usage.scala
2,163314316,test/files/pos/t5638/Among.java,test/files/pos/t9291.scala
3,163314316,test/files/pos/t5638/Among.java,test/files/run/t8348.check
4,163314316,test/files/pos/t5638/Among.java,test/files/run/t8348/TableColumn.java
